In [ ]:
#Install required packages
%pip install -U langgraph langchain_community langchain_openai langsmith grandalf

In [43]:
# Environment Variable Initialization

import getpass
import os

def _set_if_undefined(var_name: str):
    """
    Set an environment variable if it is not already defined.
    
    Args:
        var_name (str): Name of the environment variable to set.
    """
    if not os.environ.get(var_name):
        # Securely prompt the user for input without echoing it on screen
        os.environ[var_name] = getpass.getpass(f"Please provide your {var_name}: ")

# ---- Environment Variables Required ----

_set_if_undefined("OPENAI_API_KEY")         # API key for OpenAI models
_set_if_undefined("LANGSMITH_TRACING")      # Enable LangSmith tracing ("true" to enable)
_set_if_undefined("LANGSMITH_API_KEY")      # API key for LangSmith platform
_set_if_undefined("MODEL")                  # Model name (e.g., "gpt-4.1" "gpt-4o", "gpt-3.5-turbo")

In [44]:
# Lift Controller with LangGraph:
# - A multi-step, multi-agent workflow for managing elevator (lift) requests.
# - The controller determines the closest lift and routes it to the requesting floor.
# - Steps include requesting location, selecting lift, and dispatching it.

# ---- Imports ----

import os
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, END
from langgraph.types import Command

# ---- LLM Setup ----

# Load model from environment
default_model = os.environ["MODEL"]

# Initialize ChatOpenAI
llm = ChatOpenAI(model=default_model)

# ---- Configuration ----

lifts = ["lift1", "lift2"]
floors = ["floor1", "floor2", "floor3", "floor4", "floor5"]
options = lifts + [END]

# ---- State Definitions ----

class OverallState(MessagesState):
    """Graph state with current step and selected lift."""
    currentstep: Literal["1", "2", "3", END]
    selectedlift: Literal[*lifts]

class NextStep(TypedDict):
    """Output format for the controller to decide next step."""
    step: Literal["1", "2", "3", END]

class SelectedLift(TypedDict):
    """Output format for the selected lift."""
    lift: Literal[*lifts]

class LiftLocations(TypedDict):
    """Current floor location of each lift."""
    lift1loc: str
    lift2loc: str

# ---- System Prompt ----

controller_system_prompt = (
    f"You are a lift controller managing a building with {len(floors)} floors and {len(lifts)} lifts. "
    "Each floor has a call button that summons a lift. Lifts are stationed at various floors. "
    "Your goal is to choose the most appropriate lift based on proximity.\n\n"

    "# Instructions:\n"
    "1. Ask all lifts to report their current location.\n"
    "2. Analyze their positions and determine which is closest to the requesting floor. If there is a tie choose any one.\n"
    "3. Send a reservation to the selected lift and rejection notices to the others.\n\n"

    "# Example:\n"
    "- If Lift1 is on floor 1 and Lift2 on floor 5, and the user is on floor 2:\n"
    "- Lift1 is closer (|2 - 1| = 1 vs |2 - 5| = 3).\n"
    "- So, select Lift1.\n\n"

    "Always follow these steps precisely and never skip to the next step without completion of the previous one.\n"
    "Now, analyze the message history to determine the next step."
)

# ---- Controller Node ----

def lift_controller_node(state: OverallState) -> Command[Literal[*lifts, END]]:
    """Main controller logic: decide next step and route accordingly."""
    messages = [
        {"role": "system", "content": controller_system_prompt},
    ] + state["messages"] + [
        "Based on the message history, and the lift request handling process, what should be the next step? make sure all 3 steps are executed. If complete, respond with -1."
    ]

    response = llm.with_structured_output(NextStep).invoke(messages)
    nextstep = response["step"]

    if nextstep == "1":
        print("************** Step 1: Request Locations ***************")
        return Command(
            goto=["lift1", "lift2"],
            update={
                "messages": [HumanMessage(content="Step 1 -> CFP: Share your current location", name="lift_controller")],
                "currentstep": nextstep
            }
        )

    elif nextstep == "2":
        print("************** Step 2: Analyze and Select Lift ***************")

        loc_request = [
            {"role": "system", "content": controller_system_prompt}
        ] + state["messages"] + [
            "Based on the message history, identify lift1 and lift2 current locations."
        ]

        locations = llm.with_structured_output(LiftLocations).invoke(loc_request)
        lift1loc, lift2loc = locations["lift1loc"], locations["lift2loc"]
        print("Lift locations:", locations)

        reasoning_request = state["messages"] + [
            f"Given lift1 is at {lift1loc} and lift2 is at {lift2loc}, which is closer to the user's floor?"
        ]
        reasoning = llm.invoke(reasoning_request).content

        selected_lift = llm.with_structured_output(SelectedLift).invoke(reasoning_request)["lift"]
        print("Selected lift:", selected_lift)

        return Command(
            goto=["lift1", "lift2"],
            update={
                "messages": [HumanMessage(content=f"Step 2 -> Reasoning: {reasoning} Selected lift: {selected_lift}", name="lift_controller")],
                "currentstep": nextstep
            }
        )

    elif nextstep == "3":
        print("************** Step 3: Dispatch Selected Lift ***************")
        selected_lift = llm.with_structured_output(SelectedLift).invoke(messages)["lift"]
        return Command(
            goto=[selected_lift],
            update={
                "messages": [HumanMessage(content=f"Step 3 -> Selection: You ({selected_lift}) are selected", name="lift_controller")],
                "currentstep": nextstep
            }
        )

    print("************** END ***************")
    return Command(goto=END)

In [45]:
# ---- Lift Agent Nodes ----

def lift1_node(state: OverallState) -> Command[Literal["lift_controller"]]:
    """Behavior of Lift1 based on current step."""
    response_map = {
        "1": "1",  # Location
        "2": "Acknowledge",
        "3": "Moving to target floor"
    }
    return Command(
        update={
            "messages": [HumanMessage(content=response_map[state["currentstep"]], name="lift1")]
        },
        goto="lift_controller",
    )

def lift2_node(state: OverallState) -> Command[Literal["lift_controller"]]:
    """Behavior of Lift2 based on current step."""
    response_map = {
        "1": "5",  # Location
        "2": "Acknowledge",
        "3": "Moving to target floor"
    }
    return Command(
        update={
            "messages": [HumanMessage(content=response_map[state["currentstep"]], name="lift2")]
        },
        goto="lift_controller",
    )


In [48]:
# ---- Graph Assembly ----

builder = StateGraph(MessagesState)
builder.set_entry_point("lift_controller")
builder.add_node("lift_controller", lift_controller_node)
builder.add_node("lift1", lift1_node)
builder.add_node("lift2", lift2_node)

graph = builder.compile()

In [49]:
# ---- Visualization ----
graph.get_graph().print_ascii()



                +-----------+                   
                | __start__ |                   
                +-----------+                   
                       *                        
                       *                        
                       *                        
              +-----------------+               
              | lift_controller |               
              +-----------------+               
             ...       .       ...              
          ...          .          ...           
        ..             .             ..         
+-------+         +-------+         +---------+ 
| lift1 |         | lift2 |         | __end__ | 
+-------+         +-------+         +---------+ 


In [53]:
for s in graph.stream(
    {"messages": [("user", "floor 4")]}, debug=True):
    print(s)
    print("============================")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'floor 4')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'floor 4')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='floor 4', additional_kwargs={}, response_metadata={}, id='30f39456-a2cb-456c-a94c-ad94a82df509')]}
[1:tasks] Starting 1 task for step 1:
- lift_controller -> {'messages': [HumanMessage(content='floor 4', additional_kwargs={}, response_metadata={}, id='30f39456-a2cb-456c-a94c-ad94a82df509')]}
************** Step 1: Request Locations ***************
[1:writes] Finished step 1 with writes to 2 channels:
- messages -> [HumanMessage(content='Step 1 -> CFP: Share your current location', additional_kwargs={}, response_metadata={}, name='lift_controller')]
- currentstep -> '1'
[1:checkpoint] State at the end of step 1:
{'currentstep': '1',
 'messages': [HumanMessage(content='floor